Working on Second version of the model

In [24]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
from sklearn.metrics import classification_report
import numpy as np

In [5]:
import kagglehub

#Download latast version
path = kagglehub.dataset_download("sumn2u/garbage-classification-v2")

print("Path to dataset files:", path)

100%|██████████| 782M/782M [00:09<00:00, 82.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/sumn2u/garbage-classification-v2/versions/10


In [6]:
data_dir = "/root/.cache/kagglehub/datasets/sumn2u/garbage-classification-v2/versions/10"

classes = os.listdir(data_dir)
print(classes)

['trash', 'battery', 'metal', 'clothes', 'biological', 'paper', 'cardboard', 'shoes', 'plastic', 'glass']


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [8]:
#ImageNet stats (MobileNetV3 was pretrained on ImageNet)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
input_size = 224

#Define transforms (resize small images, normalize)
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(input_size, scale=(0.8,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [ ]:
#Create datasets with different transforms
train_dataset_full = datasets.ImageFolder(
    data_dir,
    transform=train_transforms
)

val_dataset_full = datasets.ImageFolder(
    data_dir,
    transform=val_transforms
)

#Split into train and val (80% train, 20% val)
val_percent = 0.2
val_size = int(len(train_dataset_full) * val_percent)
train_size = len(train_dataset_full) - val_size

generator = torch.Generator().manual_seed(42)

train_dataset, _ = random_split(
    train_dataset_full,
    [train_size, val_size],
    generator=generator
)

_, val_dataset = random_split(
    val_dataset_full,
    [train_size, val_size],
    generator=generator
)

In [ ]:
# full_dataset = datasets.ImageFolder(data_dir)

# val_percent = 0.2
# val_size = int(len(full_dataset) * val_percent)
# train_size = len(full_dataset) - val_size

# generator = torch.Generator().manual_seed(42)

# train_dataset, val_dataset = random_split(
#     full_dataset,
#     [train_size, val_size],
#     generator=generator
# )

# class TransformSubset(torch.utils.data.Dataset):

#     def __init__(self, subset, transform):
#         self.subset = subset
#         self.transform = transform
    
#     def __len__(self):
#         return len(self.subset)
    
#     def __getitem__(self, idx):
#         x, y = self.subset[idx]
#         if self.transform:
#             x = self.transform(x)
#         return x, y
    
# train_dataset = TransformSubset(train_dataset, train_transforms)
# val_dataset = TransformSubset(val_dataset, val_transforms)



In [10]:
#DataLoaders
num_workers = os.cpu_count() // 2

train_loader = DataLoader(
    train_dataset, 
    batch_size=32,
    shuffle=True,
    num_workers=num_workers
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=num_workers
)

train_classes = train_dataset_full.classes
print(f"Train_dataset: {train_classes}")
val_classes = val_dataset_full.classes
print(f"Val_dataset: {val_classes}")

#Get class count
num_train_classes = len(train_dataset_full.classes)
num_val_classes = len(val_dataset_full.classes)
print(f"Number of training classes: {num_train_classes}")
print(f"Number of validation classes: {num_val_classes}")


Train_dataset: ['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']
Val_dataset: ['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']
Number of training classes: 10
Number of validation classes: 10


In [11]:
#Load pretrained MobileNetV3-Small
model = models.mobilenet_v3_small(weights='DEFAULT')

#Freeze all layers (no gradients = no training)
for param in model.parameters():
    param.requires_grad = False

#Unfreeze last layer
for param in model.features[-1:].parameters():
    param.requires_grad = True

#Replace the classifier head with a custom classifier
model.classifier = nn.Sequential(
    nn.Linear(model.classifier[0].in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, num_train_classes)
)

model = model.to(device)

#Loss and optimizer setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


100%|██████████| 9.83M/9.83M [00:00<00:00, 85.0MB/s]


In [ ]:
def train_model(model, num_epochs=10):

    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            #Ass batch-level log
            if (batch_idx + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], "
                      f"Loss: {loss.item():.4f}")
            
        train_loss = running_loss / total
        train_acc = 100 * (correct / total)

        #Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        all_preds = []
        all_labels = []

        # Per-class stats
        class_correct = torch.zeros(num_train_classes, device=device)
        class_total = torch.zeros(num_train_classes, device=device)

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)
                all_preds.append(predicted.cpu())
                all_labels.append(labels.cpu())

                for i in range(labels.size(0)):
                    label = labels[i]
                    class_total[label] += 1
                    if predicted[i] == label:
                        class_correct[label] += 1

        val_loss /= val_total
        val_acc = 100 * (val_correct / val_total)

        scheduler.step()

        print(f"Epoch {epoch+1}/{num_epochs} | "
              f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")
        
        # Per-class accuracy
        for cls in range(num_train_classes):
            cls_mask = labels == cls
            class_total[cls] += cls_mask.sum()
            class_correct[cls] += (predicted[cls_mask] == cls).sum()
            acc = 100 * (class_correct[cls] / class_total[cls])
            print(f"    Class {train_classes[cls]}: {acc:.2f}%")

        all_preds = torch.cat(all_preds).numpy()
        all_labels = torch.cat(all_labels).numpy()

        print(classification_report(
            all_labels,
            all_preds,
            target_names=train_classes
        ))

        

In [31]:
#Train
train_model(model, num_epochs=10)

#Save model
torch.save(model.state_dict(), "mobilenetv3_waste.pth")

Epoch [1/10], Batch [10/506], Loss: 0.2258
Epoch [1/10], Batch [20/506], Loss: 0.3744
Epoch [1/10], Batch [30/506], Loss: 0.4937
Epoch [1/10], Batch [40/506], Loss: 0.2834
Epoch [1/10], Batch [50/506], Loss: 0.4103
Epoch [1/10], Batch [60/506], Loss: 0.1328
Epoch [1/10], Batch [70/506], Loss: 0.1536
Epoch [1/10], Batch [80/506], Loss: 0.4631
Epoch [1/10], Batch [90/506], Loss: 0.3864
Epoch [1/10], Batch [100/506], Loss: 0.5333
Epoch [1/10], Batch [110/506], Loss: 0.4808
Epoch [1/10], Batch [120/506], Loss: 0.2478
Epoch [1/10], Batch [130/506], Loss: 0.3841
Epoch [1/10], Batch [140/506], Loss: 0.3477
Epoch [1/10], Batch [150/506], Loss: 0.6945
Epoch [1/10], Batch [160/506], Loss: 0.4671
Epoch [1/10], Batch [170/506], Loss: 0.3762
Epoch [1/10], Batch [180/506], Loss: 0.3721
Epoch [1/10], Batch [190/506], Loss: 0.4370
Epoch [1/10], Batch [200/506], Loss: 0.3630
Epoch [1/10], Batch [210/506], Loss: 0.4151
Epoch [1/10], Batch [220/506], Loss: 0.3071
Epoch [1/10], Batch [230/506], Loss: 0.69